In [3]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

In [4]:
rng = np.random.default_rng(4324234)

def create_data_point(n):
    """
    Returns a random undirected adjacency matrix of dimension (n,n).
    The diagonal elements are interpreted as node attributes.
    """
    mat = rng.random((n, n))
    A = (mat + np.transpose(mat))/2
    return np.round(A, decimals=2)

A = create_data_point(3)
print(A)

[[0.36 0.52 0.27]
 [0.52 0.32 0.8 ]
 [0.27 0.8  0.92]]


In [8]:
def permute(A, permutation):

    # creation of the permuteted operator version of the operator A
    P = np.zeros((len(A), len(A)))
    for i,j in enumerate(permutation):
        P[i,j] = 1

    return P @ A @ np.transpose(P)



def perm_equivariant_embedding(A, betas, gammas):

    # node == quantum_spin
    # edge == link_weight_between_spins
    n_nodes = len(A)
    # num. of times that we repeat the quantum circuit
    n_layers = len(betas)
    
    # initialise in the plus state for every i-spin
    for i in range(n_nodes):
        qml.Hadamard(i)

    # repetition of the gates in the quantum circuit
    for l in range(n_layers):

        for i in range(n_nodes):
            for j in range(i):
                # factor of 2 due to definition of gate
                # creation of permuteted quantum circuit between two spin i and j
                qml.IsingZZ(2*gammas[l]*A[i,j], wires=[i,j]) 

        for i in range(n_nodes):
            # x-rotational gates for the i-spin or i-node 
            qml.RX(A[i,i]*betas[l], wires=i)

In [ ]:
n_qubits = 5
n_layers = 2

# wires indicates the qspin used in the circuit initialized in 0
# definition of the quantum device
dev = qml.device("lightning.qubit", wires=n_qubits)

# initialization
@qml.qnode(dev)

# definition of the output function associated with the quantum circuit
def eqc(adjacency_matrix, observable, trainable_betas, trainable_gammas):
    """Circuit that uses the permutation equivariant embedding"""
    
    perm_equivariant_embedding(adjacency_matrix, trainable_betas, trainable_gammas)
    # computation of the expression value associated with the observable
    return qml.expval(observable)

# Experiment
A = create_data_point(n_qubits)
betas = rng.random(n_layers)
gammas = rng.random(n_layers)

# PauliX(n) == \sigma^{(x)}_n
# n indicates the number of the spin on which the observable is applied
observable = qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(3)

# drawing the quantum circuit output associated with the experiment 
# with all the output troncated at the decimals
qml.draw_mpl(eqc, decimals=2)(A, observable, betas, gammas)
plt.show()

In [10]:
# output value of the experiment
result_A = eqc(A, observable, betas, gammas)
print("Model output for A:", result_A)

# permutation of only the spins+weights (or nodes+edges) trought A_perm
perm = [2, 3, 0, 1, 4]
A_perm = permute(A, perm)

# output value of non-permuteted observable
result_Aperm = eqc(A_perm, observable, betas, gammas)
print("Model output for permutation of A: ", result_Aperm)

# output value with the permutation of the spins (nodes) on which
# the observable is applied
observable_perm = qml.PauliX(perm[0]) @ qml.PauliX(perm[1]) @ qml.PauliX(perm[3])

result_Aperm = eqc(A_perm, observable_perm, betas, gammas)
print("Model output for permutation of A, and with permuted observable: ", result_Aperm)

Model output for A: 0.2472867456279559
Model output for permutation of A:  0.3257521248127677
Model output for permutation of A, and with permuted observable:  0.24728674562795633
